# The household resilience model
In this tutorial we describe how data on household income is connected to the model and how agents (currently households and government) interact. The tutorial is organized as follows:
1. Linking FIES to HHRM - creating the household agents
2. Set up for the household government interaction and the predisaster situation
3. Get a shock
4. Recovery

## 1. Linking FIES to HHRM

The goal is to create household agents from the Family Income and Expenditure Surveys (FIES), therefore the necessary information needs to be extracted from the dataset.
The datasets provided in the FIES usually covers much more information than needed in this model. The first challenge for the user is thus to extract the data that is really needed. In its current version the model assumes that a dataframe containing all households in the rows and all relevant parameters and variables in the columns. We here describe exactly which parameters and variables are needed and how they are then fed into the model. The required class that handles the data extraction is HHRegister that provides a list with Household(HH) objects and meta information on all households included.
To generate a HHRegister we only need to provide i) the file path and ii) the names of the columns for parameters and variables and link them to the exact input parameter in the constructor.

#### Table 1:

|Parameter|Name in testfile|Type|Description|formula|Identifier in HH-Class|
|---|---|---|---|---|---|
|id_col|HHID|int|An identifier e.g. the Household ID| - |hhid|
|weight_col|weight|float|Representative fraction of full popuation covered by HH| $w_{h}$ |weight|
|vul_col|vul|float|vulnerability factor of the HH [0-1]| $v_{h}$ |vul|
|income_col|income|float|Entire annual household income| $i_{h}$ |income_0|
|income_sp|income_sp|float|Entire annual household income from social transfers| $i_{sp}$ |income_sp|

In [13]:
## we will first have a look at the test input file
import sys
sys.path.append('/home/insauer/projects/WB_model/hhwb')
import pandas as pd

dummy_fies = pd.read_csv('/home/insauer/projects/WB_model/hhwb/data/test_data.csv')
dummy_fies

,HHID,weight,vul,income,income_sp
0,0,1,0.6,30000,0
1,1,1,0.4,40000,5000
2,2,1,0.4,35000,5000
3,3,1,0.2,50000,0
4,4,1,0.2,50000,0
5,5,1,0.7,20000,10000
6,6,1,0.7,20000,15000
7,7,1,0.1,48000,1000
8,8,1,0.1,60000,0
9,9,1,0.1,50000,0


In [14]:
## define a HHRegister
from hhwb.agents.hh_register import HHRegister

hh_reg = HHRegister()

## create HH agents in the register
hh_reg.set_from_csv(path='/data/test_data.csv', id_col='HHID', weight_col='weight', vul_col='vul',
                 income_col='income', income_sp='income_sp')
## get number of registered households 
hh_reg.n_hh

10

In [15]:
## get a list with all the households
all_hhs = hh_reg.hh_list

After the initial registration each household agent contains all the attributes taken directly from the household, they can be extracted from the Household object using the identifiers given in Table 1:

In [16]:
print('Household weight of HH 0: ' + str(all_hhs[0].weight))
print('Initial total income of HH 1: ' + str(all_hhs[1].income_0))
print('Initial income from social programms HH 3: ' + str(all_hhs[3].income_sp))
print('Initial consumption of HH 4: ' + str(all_hhs[4].consum_0))

Household weight of HH 0: 1.0
Initial total income of HH 1: 40000.0
Initial income from social programms HH 3: 0.0
Initial consumption of HH 4: 50000.0


In the current model version initial consumtion equals initial income.

## 2. Set up for the household government interaction and the predisaster situation

In the next step, all the households that have been registered are assigned to the government and on basis of all incomes and all the social transfer programs a flat income tax $\delta_{sp}^{tax}$ is calculated. Knowing the income tax  $\delta_{sp}^{tax}$, the initial effective capital stock $k_{eff}$ is then estimated from income $i_{h}$, tax $\delta_{sp}^{tax}$ and productivity of capital $\Pi_{k}$. This happens all by running the method Government.set_tax_rate().

The idea is to also adapt tax rates or include dynamic disaster support.

In [17]:
## set up a goverment
from hhwb.agents.government import Government
## select information
gov = Government()
gov.set_tax_rate(all_hhs)
print('Tax rate: ' +str(gov.tax_rate))
print('Total expenditure on social programs: ' + str(gov.sp_cost))
print('Total national capital stock: ' + str(gov.sp_cost))


 Found the Minimum!
 lambda =  0.12999999999999998 --> integ =  336.23934354565745
lambda =  0.13999999999999999 --> integ =  -94.59446803500717 


 Found the Minimum!
 lambda =  0.2800000000000001 --> integ =  55.271256215966744
lambda =  0.2900000000000001 --> integ =  -15.75708954456291 


 Found the Minimum!
 lambda =  0.2800000000000001 --> integ =  55.271256215966744
lambda =  0.2900000000000001 --> integ =  -15.75708954456291 


 Found the Minimum!
 lambda =  0.6000000000000003 --> integ =  6.619283985482457
lambda =  0.6100000000000003 --> integ =  -1.4710329502478228 


 Found the Minimum!
 lambda =  0.6000000000000003 --> integ =  6.619283985482457
lambda =  0.6100000000000003 --> integ =  -1.4710329502478228 


 Found the Minimum!
 lambda =  0.08 --> integ =  1045.0320758057976
lambda =  0.09 --> integ =  -89.86609699365307 


 Found the Minimum!
 lambda =  0.08 --> integ =  1045.0320758057976
lambda =  0.09 --> integ =  -89.86609699365307 


 Found the Minimum!
 lambda =  

The households have been now updated by their initial capital stock and we can have a lookinto this:

In [18]:
for hh in all_hhs:
    print('Capital stock of HH ' + str(int(hh.hhid))+': '+str(hh.k_eff_0))

Capital stock of HH 0: 99826.60391379737
Capital stock of HH 1: 116464.3712327636
Capital stock of HH 2: 99826.60391379737
Capital stock of HH 3: 166377.67318966228
Capital stock of HH 4: 166377.67318966228
Capital stock of HH 5: 33275.534637932455
Capital stock of HH 6: 16637.767318966227
Capital stock of HH 7: 156395.01279828255
Capital stock of HH 8: 199653.20782759474
Capital stock of HH 9: 166377.67318966228


The predisaster situation is now almost completed. The households get now prepared by the government for a shock, so that all know their optimal recovery rate $\lambda$:

In [19]:
print('Lambda HH 1: '+ str(all_hhs[1].lmbda) + ' per year')
print('Tau HH 1: '+ str(all_hhs[1].tau) + ' years')
all_hhs[5].plot_reco_trajec()
all_hhs[1].plot_reco_trajec()

Lambda HH 1: 0.2850000000000001 per year
Tau HH 1: 10.511341310715753 years


## 3. Get a shock

In [20]:
from hhwb.agents.shock import Shock
fld = Shock()
fld.set_random_shock(all_hhs)
fld.aff_hh

In [21]:
fld.unaff_hh

In [23]:
%matplotlib auto
fld.shock(gov, all_hhs)

Using matplotlib backend: Qt5Agg


In [ ]:
import matplotlib.pyplot as plt
plt.plot(fld.aff_hh[0].cons_reco)
plt.plot(fld.aff_hh[1].cons_reco)
plt.plot(fld.aff_hh[2].cons_reco)
plt.plot(fld.unaff_hh[0].cons_reco)
plt.plot(fld.unaff_hh[2].cons_reco)
plt.ylabel('consumption loss USD')
plt.xlabel('weeks')
plt.title('Absolute consumption losses')